# Crystal Cascade
A very simple American Pale Ale.

Pretty much the same as this [Basic Cascade](https://www.brewtoad.com/recipes/basic-cascade-584c84) I came accross after I'd started this recipe.

## Recipe

### Malt Bill:
 - Pilsner Malt - Australian 2 Row (3 - 4 EBC)
 - Traditional Ale Malt - Australian 2 Row (5 - 7 EBC)
 - Crystal Malt (40 - 60 EBC)

### Hops:
 - Cascade (4.5-7 Alpha Acids)

### Yeast:
 - BRY97 West Coast Ale Yeast

In [1]:
from ipywidgets import interact, fixed
import ipywidgets as widgets
import math

In [2]:
def make_widget(value, description):
    style = {'description_width' : '50%'}
    return widgets.FloatText(value=value, description=description, style=style)

# Create widgets
batch_size_box = make_widget(3.8, 'Batch Size (L) =')
grain_mass_box = make_widget(1.25, 'Total Grain Mass (kg) =')
trub_loss_box = make_widget(0.6, 'Trub Loss (L) =')
kettle_loss_box = make_widget(0.4, 'Kettle Loss (L) =')
boil_time_box = make_widget(1, 'Boil Time (hr) =')
bottle_size_box = make_widget(450, 'Bottle Size (mL) =')

# Create boxes to display widgets
left_box = widgets.VBox([batch_size_box,
                         grain_mass_box])
mid_box = widgets.VBox([trub_loss_box,
                        kettle_loss_box])
right_box = widgets.VBox([boil_time_box,
                          bottle_size_box])
widgets.HBox([left_box, mid_box, right_box])

In [3]:
batch_size = batch_size_box.value
grain_mass = grain_mass_box.value
trub_loss = trub_loss_box.value
kettle_loss = kettle_loss_box.value
boil_time = boil_time_box.value
bottle_size = int(bottle_size_box.value)

print(batch_size, 'L')
print(grain_mass, 'kg')
print(trub_loss, 'L')
print(kettle_loss, 'L')
print(boil_time, 'hr')
print(bottle_size, 'mL')

3.8 L
1.25 kg
0.6 L
0.4 L
1.0 hr
450 mL


### Calculate Water amounts
Need functions to take in batch size and work out the amounts all the way to strike water, inculding assumptions for losses etc.
Calculations and definitions from [Mash Hacks](https://mashhacks.com/how-to-calculate-water-volumes-for-brewing/):

- **Strike Water** - The starting amount of hot water (usually between 60C and 80C) that is used for the mash (adding grain to water).
- **Grain Absorption** - The amount of water absorbed by the grain.
- **First Runnings** - The amount of wort (sugary water) that is collected from the mash tun after your mash has finished.
- **Sparge Water** - Water that is added to the mash tun after mashing to rinse the grain of any left over sugars.
- **Pre-Boil Volume** - The amount of wort (sugary water) that is in your kettle before you start your boil.
- **Evaporation Rate** - The amount of water that is boiled off. Usually in Litres per Hour (or Gal/hr).
- **Post-Boil Volume** - The amount of wort left after you finish the boil.
- **Kettle Loss** - The amount of wort (if any) left at the bottom of the kettle that was not transferred to the fermenter.
- **Fermenter Volume** - The amount of wort you were able to get out of the kettle and put into the fermenter.
- **Trub Loss** aka *Fermenter loss* - The amount of beer (if any) at the bottom of the fermenter that was not transferred into bottles or kegs.
- **Batch Size** - The amount of beer that you were able to get out of the fermenter and put into bottles or kegs.

In [4]:
# This will be a module (file)
class constants(object):
    # Approximate rate of evaporation at boil (L/hr)
    evap_rate = 3
    # Approximate loss of volume due to grain absorption (L/kg)
    grain_abs_rate = 1
    # Efficiency of getting sugars from malt
#     eff = ?????????

In [5]:
fermenter_vol = batch_size + trub_loss
post_boil_vol = fermenter_vol + kettle_loss
pre_boil_vol = post_boil_vol + constants.evap_rate * boil_time

strike_water = 3 * grain_mass
grain_absorption = constants.grain_abs_rate * grain_mass
first_runnings = strike_water - grain_absorption

sparge_water = pre_boil_vol - first_runnings

In [6]:
print('Strike Water = ', strike_water)
print('Grain Absorption = ', grain_absorption)
print('First Runnings = ', first_runnings)

print('Sparge Water = ', sparge_water)

print('Pre-boil Volume = ', pre_boil_vol)
print('Post-boil Volume = ', post_boil_vol)
print('Fermenter volume = ', fermenter_vol)

Strike Water =  3.75
Grain Absorption =  1.25
First Runnings =  2.5
Sparge Water =  5.3
Pre-boil Volume =  7.8
Post-boil Volume =  4.8
Fermenter volume =  4.3999999999999995


### Calculate Expected Original Gravity

In [7]:
print(batch_size, 'L;', grain_mass, 'kg')

3.8 L; 1.25 kg


Mash Efficiency - from https://aussiehomebrewer.com/threads/working-out-mash-efficiency-in-metric.35490/

The H.W.E (hot water extract) value for the malt is the amount of sugar in the wort.
386 is the maximum (pure sugar) so every malt is given as a % of that.

In [8]:
def expected_original_hwe(grain_mass, malt_hwe, batch_size):
    potential_hwe = grain_mass * 386 * malt_hwe / batch_size
    
    original_hwe = 0.5 * potential_hwe
    
    return original_hwe

def hwe2gravity(hwe):
    try:
        total_hwe = 0
        for val in hwe:
            total_hwe += val
    except:
        total_hwe = hwe
    
    gravity = round(total_hwe / 1000 + 1.0, 3)
    
    return gravity

hwe2gravity([expected_original_hwe(1.015, 0.8, batch_size), expected_original_hwe(0.25, 0.75, batch_size)])

1.051

In [9]:
# Swapped strike and sparge waters
malt = {
    'Pilsner' : {
        'Mass' : 0.51,
        'HWE' : 0.8,
        'EBC' : 3.5
    },
    'Ale' : {
        'Mass' : 0.505,
        'HWE' : 0.8,
        'EBC' : 6
    },
    'Crystal' : {
        'Mass' : 0.25,
        'HWE' : 0.75,
        'EBC' : 50
    },
}
hops = {
    'Saaz' : {
        "Alpha Acids" : 5.5,
        "Mass" : [6, 6, 6, 4],
        "Times" : [60, 30, 5, 0]
    },
    
    '60m' : 6,
    '30m' : 6,
    '5m' : 6,
    'Dry' : 4
}

def alcohol_percentage(original_gravity, final_gravity):
    return (original_gravity - final_gravity) * 131.25

def standard_drinks(bottle_size, prc_vol):
    return bottle_size * (prc_vol/100) / 12.5

water_gravity = 1.003
original_gravity = 1.056
final_gravity = 1.01

prc_vol = alcohol_percentage(original_gravity, final_gravity)
std_drinks = standard_drinks(bottle_size, prc_vol)

print(f"Final alcohol percentage = {prc_vol:.2}% ABV")
print(f"{std_drinks:.2} Standard Drinks per {bottle_size}mL bottle")

Final alcohol percentage = 6.0% ABV
2.2 Standard Drinks per 450mL bottle


### Calculate IBU

http://www.backtoschoolbrewing.com/blog/2016/9/5/how-to-calculate-ibus

http://www.realbeer.com/hops/research.html

To calculate IBUs, the formula is simple:

`IBUs = decimal alpha acid utilization * mg/l of added alpha acids`

To calculate the concentration of alpha acids you add to the wort:

`mg/l of added alpha acids = decimal AA rating * grams hops * 1000 / volume of finished beer in liters`

You can look up the decimal alpha acid utilization in the utilization table below or calculate it directly using the Bigness factor and the Boil Time factor.

`decimal alpha acid utilization = Bigness factor * Boil Time factor`


The Bigness factor accounts for reduced utilization due to higher wort gravities. Use an average gravity value for the entire boil to account for changes in the wort volume.

`Bigness factor = 1.65 * 0.000125^(wort gravity - 1)`

The Boil Time factor accounts for the change in utilization due to boil time:

`Boil Time factor = (1 - e^(-0.04 * time in mins) ) / 4.15`


In [10]:
def ibu(mass_hops, batch_size, aa_rating, boil_gravity=original_gravity, boil_time=60):
    boil_time = max(0, boil_time)
    
    boil_time_factor = (1 - math.exp(-0.04 * boil_time)) / 4.15
    
    bigness_factor = 1.65 * 0.000125**(boil_gravity - 1)
    
    aa_utilization = bigness_factor * boil_time_factor
    
    aa_concentration = aa_rating / 100 * mass_hops * 1000 / batch_size
        
    return aa_utilization * aa_concentration

ibu(6, 4, 5.5, boil_time=60) + ibu(6, 4, 5.5, boil_time=30) + ibu(6, 4, 5.5, boil_time=5)

35.48246419724607

### Calculate EBC

http://beersmith.com/blog/2008/04/29/beer-color-understanding-srm-lovibond-and-ebc/

http://homebrewtechniques.com/mashing/how-do-you-calculate-beer-colour/

The calculation we use to work out the colour value for our beer is:

MCU = weight of malt (in lbs) x malt colour (in lovibond) / volume (in gallons)

(lovibond = SRM)

To convert our recipe from kg and litres into US gallons and lbs we use the following conversion factors:

1 litre = 0.264172 US gallons

1 kg = 2.20462 lbs.

SRM = 1.4922 * MCU^0.6859

EBC = 1.97 * SRM 

In [11]:
batch_size

3.8

In [12]:
def ebc(grain_mass, malt_colour, batch_size):
    return 1.97 * (1.4922 * ((grain_mass*2.2*malt_colour/1.97)/(batch_size*0.264172))**0.6859)

ebc(0.51, 3.5, 4) + ebc(0.505, 6, 4) + ebc(0.250, 50, 4)

28.33786046365833

In [13]:
from brewing import BeerRecipe

In [14]:
crystal_cascade = BeerRecipe("CrystalCascade", batch_size, trub_loss, kettle_loss, boil_time, bottle_size)

In [15]:
crystal_cascade.malt = malt
crystal_cascade.hops = hops

crystal_cascade.save_recipe('CrystalCascade.json')

In [16]:
crystal_cascade.read_recipe('CrystalCascade.json')

In [17]:
crystal_cascade.calculate_ibu(6, 4, 5.5, boil_time=60) + crystal_cascade.calculate_ibu(6, 4, 5.5, boil_time=30) + crystal_cascade.calculate_ibu(6, 4, 5.5, boil_time=5)

38.12751434288853

### Calculate Priming Sugar

`Cbeer = Cflat-beer + 0.5 * mtable-sugar / Vbeer`

Aim for: Cbeer = 2.2

Assume Cflat-beer = 0.85 (20 degrees C)

In [18]:
priming_sugar = fermenter_vol * (2.2 - 0.85) / 0.5
priming_sugar

11.879999999999999

In [19]:
crystal_cascade.priming_sugar

11.879999999999999

#### Example of adjustable property and dynamically allocated property:

In [20]:
print(crystal_cascade.bottle_size, crystal_cascade.standard_drinks)
crystal_cascade.bottle_size = 1000
print(crystal_cascade.bottle_size, crystal_cascade.standard_drinks)

450 1.7
1000 3.78


In [21]:
crystal_cascade.original_gravity

1.048